In [108]:
import ast
import base64
from datetime import datetime
import json
import os
from typing import Any, List, Optional

import boto3
from botocore.exceptions import NoCredentialsError
from IPython.display import display, Image, HTML

# LlamaIndex
from llama_index.core import Document
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex

from llama_index.core.embeddings import BaseEmbedding
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.schema import QueryBundle
from llama_index.core.tools import FunctionTool

# LlamaIndex agents
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner

# LlamaIndex LLMs
from llama_index.llms.openai import OpenAI as OpenAI_Llama

# LlamaIndex metadata filters
from llama_index.core.vector_stores.types import (
    MetadataFilters,FilterCondition
)

# LlamaIndex retrievers
from llama_index.core.retrievers import VectorIndexAutoRetriever, VectorIndexRetriever

# LlamaIndex vector stores
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.vector_stores.types import VectorStoreQuery

# Pinecone
from pinecone import Pinecone, ServerlessSpec

import pandas as pd
from tqdm import tqdm
from tqdm.autonotebook import tqdm

# Advent Calendar Day 1: How AI Agents Improve Naive Chatbots by Asking Clarifying Questions

This December, we're highlighting the limitations of simple AI chatbots in online retail and demonstrating how **AI agents** enhance customer interactions.

Each day, we'll explore a common challenge faced by naive Retrieval Augmented Generation (RAG) chatbot systems and show how AI agents overcome them. 

Todays topic is about how AI agents improve naive chatbots by asking clarifying questions.

![Cover image](images/1_dec/1_dec_cover.png)

## Introducing SoleMates

***SoleMates*** is a fictional online shoe store that we'll use as a practical example throughout this tutorial.

We'll explore interactions between customers and chatbots at SoleMates, highlighting the differences between basic chatbots and advanced AI agents.

![SoleMates Illustration](images/solemates.png)


## Today's Challenge: No Reflection - Simple Chatbots Can't Infer from Context

### Scenario

A customer initiates a chat with **SoleMates**:

**Customer:** "I need shoes for a black-tie event"

![A customer initiates a chat with SoleMates](images/1_dec/3_customer_black_tie.png)


## Load Shoe Data

Let's start by reading the SoleMates shoe dataset. This dataset contains detailed product information, such as shoe colors and heel heights, which we'll transform into embeddings and store in a cloud-based Pinecone vector database. 

In [6]:
# Load the SoleMates shoe dataset
df_shoes = pd.read_csv('data/solemates_shoe_directory.csv')

# Convert 'color_details' from string representation of a list to an actual list
df_shoes['color_details'] = df_shoes['color_details'].apply(ast.literal_eval)

# Ensure 'heels_height' is treated as a nullable integer type
df_shoes['heels_height'] = df_shoes['heels_height'].astype('Int64')

# Display the first few rows of the dataset
df_shoes.head()

,product_id,gender,category,sub_category,product_type,color,color_details,usage,product_title,image,price_usd,heels_height
0,1,men,footwear,shoes,sports shoes,black,[neon green],sports,Adidas men eqt nitro fashion black sports shoes,1.jpg,120,<NA>
1,2,men,footwear,shoes,sports shoes,black,[white],sports,Puma men's yugorun black white shoe,2.jpg,50,<NA>
2,3,men,footwear,shoes,boots,black,[],casual,Timberland men black casual shoes,3.jpg,60,<NA>
3,4,men,footwear,shoes,casual shoes,black,[],casual,Provogue men black shoes,4.jpg,125,<NA>
4,5,men,footwear,shoes,formal shoes,black,[],formal,Lee cooper men black shoe,5.jpg,155,<NA>


## Cost of Vectorization and Pre-Embedded Dataset

Vectorizing datasets with AWS Bedrock and the Titan multimodal model involves costs based on the number of input tokens and images:

- **Text embeddings**: $0.0008 per 1,000 input tokens  

- **Image embeddings**: $0.00006 per image  

The provided SoleMates dataset is small, containing just 85 pairs of shoes, making it affordable to vectorize. For this dataset, I calculated the total cost of vectorization and summarized the token counts below:

- **Token Count**: `831` tokens  
- **Total Cost**: `$0.0058`  

If you prefer not to generate embeddings yourself or don't have access to AWS, you can use a pre-embedded dataset that I've prepared as a CSV file. This file includes all embeddings and token counts, allowing you to follow the guide without incurring additional costs. However, for hands-on experience, I recommend running the embedding process to understand the workflow.

To load the pre-embedded dataset, use the following code:
```python
# Load pre-embedded dataset
df_shoes = pd.read_csv('data/solemates_shoe_directory_pre_embedded_shoes.csv')
```
This step is entirely optional and designed to accommodate various levels of access and resources.

### Prepare Amazon Bedrock for Embedding Generation

To vectorize our product data, we'll generate embeddings for each product using AWS Titan. These embeddings combine image and text data to represent each product in a format suitable for search and recommendation systems.

>**Important Note on Cost**:  
>Vectorizing datasets incurs a cost. The SoleMates dataset contains 85 pairs of shoes, resulting in an estimated total cost of `$0.0058`.
>
>I've added a token count column to help track these costs, and you can calculate your own total for larger datasets.

If you'd rather not generate embeddings yourself, you can load a pre-embedded version of the dataset I've provided. This is entirely optional but ensures you can still follow along with the guide:
```python
# Load pre-embedded dataset
df_shoes = pd.read_csv('data/solemates_shoe_directory_pre_embedded_shoes.csv')
```

## Getting started with Amazon Bedrock
To use Amazon Bedrock for embedding generation, start by setting up your AWS environment:

1. Create an AWS account if you don't already have one
2. Set up an AWS Identity and Access Management (IAM) role with permissions tailored for Amazon Bedrock
3. Submit a request to access the foundation models (FMs) you'd like to use

Next, we'll initialize the Bedrock runtime client, which allows us to interact with AWS Titan for embedding generation.


## Set up AWS Bedrock client

In [8]:
# Define your AWS profile 
# Replace 'your-profile-name' with the name of your AWS CLI profile
# To use your default AWS profile, leave 'aws_profile' as None
aws_profile = os.environ.get('AWS_PROFILE')

# Specify the AWS region where Bedrock is available
aws_region_name = "us-east-1"

try:
    # Set the default session for the specified profile
    if aws_profile:
        boto3.setup_default_session(profile_name=aws_profile)
    else:
        boto3.setup_default_session()  # Use default AWS profile if none is specified
    
    # Initialize the Bedrock runtime client
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=aws_region_name
    )
except NoCredentialsError:
    print("AWS credentials not found. Please configure your AWS profile.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

## Generate Embeddings for Product Data

To prepare our product data for the vector database, we'll generate embeddings for each product using AWS Titan. These embeddings combine image and text data to represent each product in a format suitable for search and recommendation systems.

Before generating embeddings, we'll initialize two new columns in the dataset:
- **`titan_embedding`**: To store the embedding vectors.
- **`token_count`**: To store the token count for each product title.

Then, we'll define a function to generate embeddings and apply it to the dataset.

## Initialize Columns for Embeddings

In [9]:
# Initialize columns to store embeddings and token counts
df_shoes['titan_embedding'] = None  # Placeholder for embedding vectors
df_shoes['token_count'] = None  # Placeholder for token counts

In [17]:
# Main function to generate image and text embeddings
def generate_embeddings(df, image_col='image', text_col='product_title', embedding_col='embedding', image_folder='data/footwear'):
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Generating embeddings"):
        try:
            # Prepare image file as base64
            image_path = os.path.join(image_folder, row[image_col])
            with open(image_path, 'rb') as img_file:
                image_base64 = base64.b64encode(img_file.read()).decode('utf-8')
            
            # Create input data for the model
            input_data = {"inputImage": image_base64, "inputText": row[text_col]}

            # Invoke AWS Titan model via Bedrock runtime
            response = bedrock_runtime.invoke_model(
                body=json.dumps(input_data),
                modelId="amazon.titan-embed-image-v1",
                accept="application/json",
                contentType="application/json"
            )
            response_body = json.loads(response.get("body").read())

            # Extract embedding and token count from response
            embedding = response_body.get("embedding")
            token_count = response_body.get("inputTextTokenCount")

            # Validate and save the embedding
            if isinstance(embedding, list):
                df.at[index, embedding_col] = embedding  # Save embedding as a list
                df.at[index, 'token_count'] = int(token_count)  # Save token count as an integer
            else:
                raise ValueError("Embedding is not a list as expected.")
                            
        except Exception as e:
            print(f"Error for row {index}: {e}")
            df.at[index, embedding_col] = None  # Handle errors gracefully
            
    return df

## Generate Embeddings

In [18]:
# Generate embeddings for the product data
df_shoes = generate_embeddings(df=df_shoes, embedding_col='titan_embedding')


enerating embeddings: 100%|████████████████████████████████████████████████████████| 85/85 [00:24<00:00,  3.47it/s]

## Save Dataset for Reuse

In [22]:
# Save the dataset with generated embeddings to a CSV file
# Get today's date in YYYY_MM_DD format
today = datetime.now().strftime('%Y_%m_%d')

# Save the dataset with generated embeddings to a CSV file
df_shoes.to_csv(f'shoes_with_embeddings_token_{today}.csv', index=False)
print(f"Dataset with embeddings saved as 'shoes_with_embeddings_token_{today}.csv'")

Dataset with embeddings saved as 'shoes_with_embeddings_token_2024_12_06.csv'


## Create a Dictionary with Product Data

Before we create LlamaIndex `Document` objects, we need to structure the product data into dictionaries. These dictionaries include:

1. **Text**: The product title that will be used for embedding queries.
2. **Metadata**: A dictionary containing detailed attributes for each product (e.g., color, gender, usage, price).
3. **Embedding**: The Titan embeddings generated earlier.

This dictionary format ensures the data is well-organized for creating `Document` objects in the next step.


In [ ]:
# Convert DataFrame rows into a list of dictionaries for LlamaIndex
product_data = df_shoes.apply(lambda row: {
    'text': row['product_title'],
    'metadata': {
        'color': row['color'],
        'text': row['product_title'],
        'gender': row['gender'],
        'product_type': row['product_type'],
        'usage': row['usage'],
        'price': row['price_usd'],
        'product_id': row['product_id'],
        **({'heels_height': int(row['heels_height'])} if not pd.isna(row['heels_height']) else {}),
        **({'color_details': row['color_details']} if row['color_details'] else {})
    },
    'embedding': row['titan_embedding']
}, axis=1).tolist()

# Preview the first product dictionary
#product_data[0]


## Create LlamaIndex Documents

We'll now use the product data dictionaries to create LlamaIndex `Document` objects. 

These `Documents` are crucial because:

- They act as containers for our product data and embeddings.
- They enable seamless interaction with Pinecone for upserting embeddings.

Each `Document` includes:
1. The **text** (product title) for embedding and query purposes
2. **Metadata** with attributes like color, gender, and price
3. The **embedding** generated earlier
4. An **exclusion list** (`excluded_embed_metadata_keys`) to prevent unnecessary metadata fields from being embedded, ensuring optimal performance and cost-efficiency


## Create LlamaIndex Documents

In [26]:
# Create LlamaIndex Document objects
documents = []
for doc in product_data:
    documents.append(
        Document(
            text=doc["text"],
            extra_info=doc["metadata"],
            embedding=doc['embedding'],
            
            # Avoid embedding unnecessary metadata
            excluded_embed_metadata_keys=[
                'color',
                'gender',
                'product_type',
                'usage',
                'text',
                'price',
                'product_id',
                'heels_height',
                'color_details'
            ]
        )
    )

# Confirm the first Document object
documents[0].metadata

{'color': 'black',
 'text': 'Adidas men eqt nitro fashion black sports shoes',
 'gender': 'men',
 'product_type': 'sports shoes',
 'usage': 'sports',
 'price': 120,
 'product_id': 1,
 'color_details': ['neon green']}

## Initialize Pinecone

To interact with Pinecone, you'll first need an account and API keys. If you don't already have them, [create a Pinecone account](https://www.pinecone.io/) and retrieve your API key.

Pinecone is a vector database designed to store and query embeddings. We'll use Pinecone to upsert the AWS Titan embeddings we generated earlier, enabling efficient similarity and hybrid search.


In [31]:
# Initialize Pinecone client with API key
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
index_name = "solemates"  # Replace with your desired index name

## List Current Indexes

Let's list the existing indexes in your Pinecone account to ensure no duplicates before creating a new index.

In [33]:
# List current indexes
pc.list_indexes()

{'indexes': []}

## Create Index

Next, we'll create a Pinecone index. An index stores the embeddings and metadata for your data.

- **Dimension**: Matches the size of the embeddings we're using (1024 for AWS Titan multimodal embeddings)
- **Metric**: Defines how similarity is calculated (e.g., dot product, cosine similarity)
- **ServerlessSpec**: Specifies the cloud provider and region for your index

If the index already exists, this step will be skipped

In [38]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # AWS Titan embeddings require 1024 dimensions
        metric="dotproduct",  # Required for hybrid search with Pinecone
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

## Inspect Pinecone Index

Navigate to your Pinecone dashboard, and you should now see your new index with **0 records (vectors)**, as it hasn't been populated yet:

![Pinecone shows an empty index](images/pinecone/4_pinecone_empty_index.png)


## Initialize Pinecone Index

After creating the index, we'll initialize it for further operations like upserting embeddings and querying vectors.

In [40]:
pinecone_index = pc.Index(index_name)

## Create Pinecone Vector Store

We'll now set up a **Pinecone Vector Store** using LlamaIndex. 

This vector store connects our Pinecone index with the LlamaIndex framework.

Key configuration details:
1. **Namespace**: A logical grouping within the index, allowing future addition of other product types.
2. **Hybrid Search**: Enabling both semantic and keyword search by adding sparse vectors.

For more information:
- [Pinecone Namespaces Guide](https://docs.pinecone.io/guides/indexes/use-namespaces)
- [Hybrid Search Introduction](https://www.pinecone.io/learn/hybrid-search-intro/)


In [46]:
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
    namespace='footwear',  # Logical namespace for shoe data
    add_sparse_vector=True  # Enables hybrid search
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## Create an Ingestion Pipeline

We'll create an **Ingestion Pipeline** to upsert our vectors into the Pinecone index. 
No transformations are required since we've pre-generated embeddings with AWS Titan.

>**Note**: As of Dec 4 2024, LlamaIndex doesn't abstract AWS Titan multimodal embeddings, so we're using our own vectors directly.


In [49]:
pipeline = IngestionPipeline(
    transformations=[],  # No transformations since embeddings are pre-generated
    vector_store=vector_store
)

## Run the Ingestion Pipeline

This step upserts the embeddings into Pinecone for storage and querying.

- **Cost Note**: Pinecone charges $2.00 per 1M vectors unless you're on the free plan
- **Time Note**: It may take a minute or two for the vectors to become visible in your Pinecone index


In [ ]:
# Run the pipeline to upsert embeddings into Pinecone
pipeline.run(documents=documents, show_progress=True)

## Inspect Pinecone Index

Now that we've upserted the vectors, navigate back to Pinecone. You should see **85 records** in your index, corresponding to the embeddings we added:

![Populated Pinecone index](images/pinecone/5_pinecone_populated_index.png)


## Test Querying the Vector Store

Now that we have upserted all our shoe vectors, let's test querying the vector database.  

We'll start by creating a **Vector Store Index** with LlamaIndex. This index will allow us to query the Pinecone index using the same vector store we initialized earlier.


In [53]:
# Create a Vector Store Index
vector_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

## Query the Vector Database Directly (Without Query Engine or Chat Engine)

Before we use a **Query Engine** or **Chat Engine** to interact with the vector database, we'll start with a direct query using a simple retriever.  

This approach demonstrates how you can fetch relevant records from the database without involving advanced reasoning, natural language understanding, or conversation tracking. It's a fundamental way to confirm that the embeddings and metadata are stored correctly and the vector database is functioning as expected.

Next, we'll move on to more advanced querying techniques, including using a **Query Engine** and an **Agent** to leverage the power of LLMs.

The first step is creating a simple retriever, but first, we need to define a custom embedding function. 

As of Dec 4, 2024, **LlamaIndex does not abstract AWS Titan multimodal embeddings**, so we'll implement a custom class for this purpose.

## Create a Function to Request AWS Titan Embeddings

We'll define a helper function to request embeddings from AWS Titan's multimodal model. This function will handle both text and image inputs.


In [54]:
def request_embedding(image_base64=None, text_description=None):
    """
    Request embeddings from AWS Titan multimodal model.

    Parameters:
        image_base64 (str, optional): Base64 encoded image string.
        text_description (str, optional): Text description.

    Returns:
        list: Embedding vector.
    """
    input_data = {"inputImage": image_base64, "inputText": text_description}
    body = json.dumps(input_data)

    # Invoke the Titan multimodal model
    response = bedrock_runtime.invoke_model(
        body=body,
        modelId="amazon.titan-embed-image-v1",
        accept="application/json",
        contentType="application/json"
    )

    response_body = json.loads(response.get("body").read())

    if response_body.get("message"):
        raise ValueError(f"Embeddings generation error: {response_body.get('message')}")

    return response_body.get("embedding")


## Create Custom Embeddings Class

We'll now define a custom embedding class that uses the AWS Titan multimodal model to fetch embeddings. 

This class overrides key methods in LlamaIndex's `BaseEmbedding` to integrate AWS Titan into the framework.


In [58]:
class MultimodalEmbeddings(BaseEmbedding):
    """
    Custom embedding class for AWS Titan multimodal embeddings.
    """

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)

    @classmethod
    def class_name(cls) -> str:
        return "multimodal"
    
    async def _aget_query_embedding(self, query: str) -> List[float]:
        return self._get_query_embedding(query)

    async def _aget_text_embedding(self, text: str) -> List[float]:
        return self._get_text_embedding(text)

    def _get_query_embedding(self, query: str) -> List[float]:
        """
        Get embeddings for a query string.
        """
        return request_embedding(text_description=query)

    def _get_text_embedding(self, text: str) -> List[float]:
        """
        Get embeddings for a text string.
        """
        return request_embedding(text_description=text)

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        """
        Get embeddings for a batch of text strings.
        """
        return [request_embedding(text_description=text) for text in texts]


## Instantiate the Custom Class

We'll now instantiate the `MultimodalEmbeddings` class to use it in our retriever.


In [59]:
# Instantiate the custom embedding model
embed_model = MultimodalEmbeddings()

## Create a Simple Retriever

We'll create a simple retriever using the custom embedding model and the vector index.  

Key configurations:
1. **`similarity_top_k`**: Number of top results to retrieve
2. **`vector_store_query_mode`**: Set to **"hybrid"** for combining semantic and keyword search
3. **`alpha`**: Weighting between semantic (embedding) and keyword search


## Create Retriever

In [62]:
# Create a simple retriever
retriever = VectorIndexRetriever(
    index=vector_index,
    embed_model=embed_model,
    similarity_top_k=8,  # Retrieve the top 8 results
    vector_store_query_mode="hybrid",  # Enable hybrid search
    alpha=0.5  # Weighting between semantic and keyword search
)

## Query Vector Database Directly Using a Simple Retriever

We'll use a simple retriever to query the vector database and inspect the results. This method interacts with the embeddings and metadata in a straightforward way, without utilizing an LLM-powered **Query Engine** or **Chat Engine**.

**Why this step matters**:
1. Validates that the vector database is populated correctly
2. Shows how to query embeddings directly, bypassing the overhead of LLM-based reasoning
3. Prepares the groundwork for building advanced workflows with Query Engines and Agents

In the next steps, we'll extend this retriever to integrate with an LLM-powered Query Engine for richer responses.


## Query the Vector Store

In [63]:
# Query the vector store for "red shoes"
results = retriever.retrieve("red shoes")

# Display results
for item in results:
    score = item.score
    print(f"Score: {score:.4f}")
    print(f"Text: {item.get_content()}")
    print("-" * 50)

Score: 2.2395
Text: Catwalk women red shoes
--------------------------------------------------
Score: 2.2250
Text: Fila men leonard red shoes
--------------------------------------------------
Score: 2.2233
Text: Basics men red casual shoes
--------------------------------------------------
Score: 2.2148
Text: Carlton london women casual red casual shoes
--------------------------------------------------
Score: 2.2105
Text: Nike men jordan fly wade red sports shoes
--------------------------------------------------
Score: 2.1867
Text: Red tape men brown shoes
--------------------------------------------------
Score: 2.1822
Text: Adidas men blue & red f10 sports shoes
--------------------------------------------------
Score: 2.1681
Text: Red tape men casual brown casual shoes
--------------------------------------------------


## Visualize Vector Database Pull

The vector database query returns a list of red shoes based on the embeddings. To verify the results, let's visualize the pulled vectors.  

We'll create a function that loops through the retrieved nodes and displays each image along with its metadata in a row for easy inspection.


In [69]:
def display_nodes_with_images_in_row(vector_database_response_nodes, image_folder="data/footwear", img_width=150):
    html_content = "<div style='display: flex; flex-wrap: wrap; gap: 20px;'>"
    
    for node in vector_database_response_nodes:
        # Retrieve text and product_id from node metadata
        text = node.metadata.get('text')
        product_id = node.metadata.get('product_id')
        
        # Generate image path based on product_id
        image_path = os.path.join(image_folder, f"{product_id}.jpg")
        
        if os.path.exists(image_path):
            # Add each text and image in a flex container
            html_content += f"""
                <div style="text-align: center;">
                    <p>{text}</p>
                    <img src='{image_path}' width='{img_width}px' style="border: 1px solid #ddd; padding: 5px;"/>
                </div>
            """
        else:
            # Handle missing images gracefully
            html_content += f"""
                <div style="text-align: center;">
                    <p>{text}</p>
                    <p style='color: red;'>Image not found for product_id {product_id}</p>
                </div>
            """

    # Close the main div
    html_content += "</div>"
    
    # Display the content as HTML
    display(HTML(html_content))

## Visualize Shoes
Let's visualize the shoes retrieved from the vector database to confirm that the results match the query for "red shoes."

In [70]:
display_nodes_with_images_in_row(results)

## Examine the Shoes

As shown, all the retrieved shoes are red or have red details, confirming that the vector index query works well for focused queries.  

Next, we'll involve an LLM to add more flexibility to our queries.


## Why Not Keep Using the Index Alone?

The results look great, so why involve an LLM?

Let's try querying the vector database with something unrelated, like "Thank you!" and examine the response.

![Why involve an LLM?](images/1_dec/6_naive_rag_reply.png)


## Test Naive Query

In [71]:
# Query the vector store with an unrelated query
results = retriever.retrieve("Thank you!")

# Display results
for item in results:
    score = item.score
    print(f"Score: {score:.4f}")
    print(f"Text: {item.get_content()}")
    print("-" * 50)

# Visualize the results
display_nodes_with_images_in_row(results)


Score: 1.1846
Text: Timberland women femmes brown boot
--------------------------------------------------
Score: 1.1784
Text: Nike men's air max black shoe
--------------------------------------------------
Score: 1.1670
Text: Nike men's egoli white black shoe
--------------------------------------------------
Score: 1.1652
Text: Nike women ten blue white shoe
--------------------------------------------------
Score: 1.1650
Text: Nike women zoo blue shoe
--------------------------------------------------
Score: 1.1609
Text: Adidas women's piona white shoe
--------------------------------------------------
Score: 1.1591
Text: Nike women main draw white blue shoe
--------------------------------------------------
Score: 1.1566
Text: Hm women brown shoes
--------------------------------------------------


## Limitations of a Naive RAG System

Regardless of the query, the vector database matches the closest vectors based on the embeddings. 

In this case, querying with "Thank you!" still returns shoes because the database doesn't understand the context or intent of the query.

This demonstrates the limitation of a **naive RAG (Retrieval-Augmented Generation) system**. 

While it works well for focused queries like **"red shoes"**, it fails to adapt to non-specific or conversational inputs.

Here's an illustration of this limitation:

![Naive RAG System](images/1_dec/6_naive_rag.png)

## Create a Vector Index Query Engine

To overcome the limitations of naive queries, we'll integrate an LLM into our workflow by creating a **Query Engine**.  

This Query Engine will:
1. Interpret the user's natural language input
2. Retrieve contextually relevant information from the vector database
3. Enable more dynamic and flexible interactions with the data

For this guide, we'll use the `openai-o4` model for the LLM.


## Initialize LLM

In [78]:
# Initialize LLM
llm = OpenAI_Llama(
    temperature=0.0, 
    model="gpt-4o", 
    api_key=os.environ["OPENAI_API_KEY"]
)
Settings.llm = llm

## Create Query Engine

We'll now create a Query Engine using the vector index and our custom embedding model. This engine will leverage the LLM for intelligent query interpretation and responses.


In [79]:
# Create a query engine from the vector index
query_engine = vector_index.as_query_engine(
    embed_model=embed_model,
    similarity_top_k=8,
    vector_store_query_mode="hybrid",
    alpha=0.5,
)

## Test with Today's Challenge: Shoes for a Black-Tie Event

Let's test the Query Engine by asking for shoes suitable for a black-tie event.


In [80]:
# Query the engine for black-tie event shoes
response = query_engine.query("I need shoes for a black-tie event")

print("Chatbot response: ", response.response)

Chatbot response:  For a black-tie event, you would typically need formal shoes. The options provided are all casual or sports shoes, which may not be suitable for such an occasion.


## Visualize pulled shoes

In [81]:
display_nodes_with_images_in_row(response.source_nodes)

## Examine Results

Looking at the results, the chatbot correctly understands the context of "black-tie event" as requiring formal shoes.  

However, while the LLM correctly identifies that formal shoes are needed, it seems to retrieve casual or sports shoes due to the vector query. Here's the chatbot reply:

> "For a black-tie event, you would typically need formal shoes. The options provided are casual or sports shoes, which may not be suitable for such an occasion."


### Why Did the Naive Chatbot Fail?

The naive chatbot struggled with this query because it's designed to vectorize the **entire user query** and match it against the vector database. While this approach works for straightforward searches (e.g., "red shoes"), it has limitations for nuanced queries like "shoes for a black-tie event."  

**Key Limitations**:
1. **Forced Query Vectorization**: By embedding the entire user query, the system treats every word in the query as equally important. This biases the search towards keywords like "shoes" or "black," overlooking the broader intent of "black-tie event."
2. **Lack of Contextual Understanding**: The vector database operates without reasoning or context, so it retrieves products based solely on similarity to the query embedding, even if the retrieved results don't match the user's intent.

In this case, the LLM recognized the mismatch (e.g., pulling casual or sports shoes for a formal event) but couldn't directly address it because the retrieval mechanism didn't align with the nuanced query.  

Despite that black formal shoes exist in the dataset, the system failed to retrieve them:

In [82]:
# Verify the dataset for black formal shoes
df_shoes[(df_shoes['product_type'] == 'formal shoes') & (df_shoes['color'] == 'black')]

,product_id,gender,category,sub_category,product_type,color,color_details,usage,product_title,image,price_usd,heels_height,titan_embedding,token_count
4,5,men,footwear,shoes,formal shoes,black,[],formal,Lee cooper men black shoe,5.jpg,155,<NA>,"[-0.0017635934, -0.009471155, -0.014912436, -0...",7
5,6,men,footwear,shoes,formal shoes,black,[],formal,Arrow men formal black shoe,6.jpg,180,<NA>,"[0.013223984, -0.018667577, -0.024809424, -0.0...",8


## Query engine limitations

This demonstrates another limitation: when the query embedding does not align perfectly with the database, important results may be missed.

To address this, we'll now involve an AI Agent to enhance query handling.

## Create Vector Store Info

We'll define metadata about the vector store to allow the AI agent to filter results based on specific attributes like gender, usage, and color. This metadata will enhance the agent's ability to refine queries.


In [85]:
# Create vector store information
vector_store_info = VectorStoreInfo(
    content_info="shoes in the shoe store",
    metadata_info=[
        MetadataInfo(
            name="gender",
            type="str",
            description="Either 'men' or 'women'",
        ),
        MetadataInfo(
            name="usage",
            type="str",
            description="Either 'sports', 'casual', or 'formal'",
        ),
        MetadataInfo(
            name="color",
            type="str",
            description=("Either 'black', 'white', 'blue', 'turquoise blue', 'red', 'pink', 'brown', 'green', or 'multi'"),
        ),
    ],
)


## Create Tools

Tools are essential for enabling the AI Agent to interact with the vector store.  
We'll define two tools:
1. **`create_metadata_filter`**: Generates metadata filters for refining the search query
2. **`search_footwear_database`**: Searches the vector database using the query and optional filters


## Define Metadata Filter Tool

In [109]:
# Define a tool to create metadata filters
def create_metadata_filter(filter_string):
    """
    Creates metadata filter JSON for vector database queries.

    Args:
        filter_string (str): Query string for generating metadata filters.

    Returns:
        str: JSON string of filters.
    """
    class CustomRetriever(VectorIndexAutoRetriever):
        def __init__(self, vector_index, vector_store_info, **kwargs):
            super().__init__(vector_index, vector_store_info, **kwargs)

        def _retrieve(self, query, **kwargs):
            query_bundle = QueryBundle(query_str=query)
            retrieval_spec = self.generate_retrieval_spec(query_bundle)
            return retrieval_spec

    custom_retriever = CustomRetriever(vector_index=vector_index, vector_store_info=vector_store_info)
    retrieval_spec = custom_retriever._retrieve(filter_string)

    filters_dicts = [{'key': f.key, 'value': f.value, 'operator': f.operator.value} for f in retrieval_spec.filters]
    return json.dumps(filters_dicts)


## Define Footwear Vector Database Search Tool

In [110]:
# Define a tool to search the footwear vector database
def search_footwear_database(query_str, filters_json=None):
    """
    Searches the footwear vector database using a query string and optional filters.

    Args:
        query_str (str): Query string describing the footwear.
        filters_json (Optional[List]): JSON list of metadata filters.

    Returns:
        list: Search results from the vector database.
    """

    # Generate the embedding for the query string
    query_embedding = embed_model._get_query_embedding(query_str)

    # Deserialize from JSON
    metadata_filters = MetadataFilters.from_dicts(filters_json, condition=FilterCondition.AND)
    
    vector_store_query = VectorStoreQuery(
        query_str=query_str,
        query_embedding=query_embedding,
        alpha=0.5,
        mode='hybrid',
        filters=metadata_filters,
        similarity_top_k=10
    )
    
    # Execute the query against the vector store
    query_result = vector_store.query(vector_store_query)

    # Create output without embeddings
    nodes_with_scores = []
    for index, node in enumerate(query_result.nodes):
        score: Optional[float] = None
        if query_result.similarities is not None:
            score = query_result.similarities[index]
        nodes_with_scores.append({
            'color': node.metadata['color'],
            'text': node.metadata['text'],
            'gender': node.metadata['gender'],
            'product_type': node.metadata['product_type'],
            'product_id': node.metadata['product_id'],
            'usage': node.metadata['usage'],
            'price': node.metadata['price'],
            'similarity_score': score
        })

    return nodes_with_scores


## Define Agent Tools

In [111]:
create_metadata_filters_tool = FunctionTool.from_defaults(
    name="create_metadata_filter",
    fn=create_metadata_filter
)

query_vector_database_tool = FunctionTool.from_defaults(
    name="search_footwear_database",
    fn=search_footwear_database
)

## Create AI Agent

We'll now define an AI Agent capable of reasoning over the data, generating filters, and performing refined searches to address customer queries more effectively.


### Create the agent worker

In [112]:
# Create the agent worker
agent_worker = FunctionCallingAgentWorker.from_tools(
    [
        create_metadata_filters_tool,
        query_vector_database_tool,
    ],
    llm=llm,
    verbose=True,
    system_prompt="""\
You are an agent designed to answer customers looking for shoes.\
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\
Drive sales and always feel free to ask a user for more information.\

- Always consider if filters are needed based on the user's query.
- Use the tools provided to answer questions; do not rely on prior knowledge.

**Example:**

User Query: "Hi! I'm going to a party and I'm looking for red women's shoes. Thank you!"

Agent Actions:

1. Determine what query string to use for filters e.g. "red woman's shoes"
2. Call:
   filter_string = create_metadata_filter_string("red woman's shoes")
3. Call:
   results = search_footwear_database(query_str='shoes', filter_string=filter_string)

Remember to follow these instructions carefully.
""",
)

### Create the agent runner

In [113]:
agent = AgentRunner(agent_worker)

## Test Agent

Let's test the AI Agent by asking for shoes suitable for a black-tie event.


In [114]:
# Test the agent
response = agent.chat("I need shoes for a black-tie event")

Added user message to memory: I need shoes for a black-tie event
=== LLM Response ===
To help you find the perfect shoes for a black-tie event, could you please specify if you are looking for men's or women's shoes? Additionally, do you have any preferences for color or style?


### Interpreting the Query

Unlike a naive query engine, the AI agent approaches the request by interpreting the context and inferring the customer's true intent.  

For the query **"I need shoes for a black-tie event"**, the agent doesn't immediately return a fixed set of vectors. Instead, it evaluates the context of the request and asks clarifying questions to ensure a precise recommendation.  

Here's the agent's initial response:
> **"To help you find the perfect shoes for a black-tie event, could you please specify if you are looking for men's or women's shoes? Additionally, do you have any preferences for color or style?"**

By doing so, the agent ensures it fully understands the customer's needs before proceeding with a search:

![Agent ensures it fully understands the customer's needs](images/1_dec/6_agent_clarification_question.png)


In [115]:
# Specify preferences in the query
agent_response = agent.chat("Men's shoes for a black-tie event")

Added user message to memory: Men's shoes for a black-tie event
=== Calling Function ===
Calling function: create_metadata_filter with args: {"filter_string": "men's black-tie shoes"}
=== Function Output ===
[{"key": "gender", "value": "men", "operator": "=="}]
=== Calling Function ===
Calling function: search_footwear_database with args: {"query_str": "black-tie shoes", "filters_json": [{"key": "gender", "value": "men", "operator": "=="}]}
=== Function Output ===
[{'color': 'black', 'text': 'Provogue men black shoes', 'gender': 'men', 'product_type': 'casual shoes', 'product_id': 4, 'usage': 'casual', 'price': 125, 'similarity_score': 2.22137713}, {'color': 'black', 'text': 'Timberland men black casual shoes', 'gender': 'men', 'product_type': 'boots', 'product_id': 3, 'usage': 'casual', 'price': 60, 'similarity_score': 2.20507097}, {'color': 'black', 'text': 'Adidas men eqt nitro fashion black sports shoes', 'gender': 'men', 'product_type': 'sports shoes', 'product_id': 1, 'usage': 's

# Detailed Agent Workflow

## Agent Workflow

When refining the query to "Men's shoes for a black-tie event," the agent takes the following steps:

1. **Contextual Understanding**:
   - Recognizes the query's requirement for formal men's shoes suitable for a black-tie event.
2. **Tool Invocation**:
   - Calls the `create_metadata_filter` function with the argument `filter_string="men's black-tie shoes"`, generating a metadata filter:
     ```json
     [{"key": "gender", "value": "men", "operator": "=="}]
     ```
   - Calls the `search_footwear_database` function with the query string "black-tie shoes" and the generated filter. This refines the search to include only men's shoes.
3. **Response Generation**:
   - Combines the retrieved vector database results with its contextual understanding to provide a natural language response.

Here's the agent's final response:
> "Here are some options for men's shoes suitable for a black-tie event:  
>  1. **Arrow Men Formal Black Shoe**  
>    - Color: Black  
>    - Type: Formal Shoes  
>    - Price: \$180
>  
>  2. **Enroute Men Leather Brown Formal Shoes**  
>    - Color: Brown  
>    - Type: Formal Shoes  
>    - Price: \$70
>  
> These formal shoes would be appropriate for a black-tie event. If you have any specific preferences or need further assistance, feel free to let me know!"

This workflow highlights the agent's ability to bridge gaps between the customer's query and the vector database results by applying reasoning and metadata filters.

![Agent workflow](images/1_dec/6_agent_clarification.png)

In [116]:
recommended_shoes = ['Arrow Men Formal Black Shoe', 'Enroute Men Leather Brown Formal Shoes']

# Visualize the Agent's Recommendations

In [118]:
image_folder="data/footwear"
img_width=150
html_content = "<div style='display: flex; flex-wrap: wrap; gap: 20px;'>"

for node in agent_response.sources[1].raw_output:

    if any(node['text'].lower() in shoe.lower() for shoe in recommended_shoes):
        # Generate image path based on product_id
        image_path = os.path.join(image_folder, f"{node['product_id']}.jpg")
        html_content += f"""
            <div style="text-align: center;">
                <p>{node['text']}</p>
                <img src='{image_path}' width='{img_width}px' style="border: 1px solid #ddd; padding: 5px;"/>
            </div>
        """
# Close the main div
html_content += "</div>"

# Display the content as HTML
display(HTML(html_content))

## Why the AI Agent Succeeds

This example illustrates how the AI agent overcomes the limitations of naive RAG systems and simple LLM-powered query engines:

1. **Clarifies Ambiguous Queries**:  
   By asking follow-up questions, the agent ensures it understands the user's preferences (e.g., gender, style) before proceeding.
   
2. **Refines Search with Metadata Filters**:  
   It generates and applies metadata filters to focus the search results on the most relevant subset of the vector database. This step bridges the gap between broad vector queries and the customer's specific intent.

3. **Combines Contextual Reasoning with Data**:  
   The agent synthesizes its understanding of "black-tie event" with the vector database results to provide accurate and human-like recommendations.

The AI agent's workflow highlights the value of integrating reasoning, filtering, and retrieval for advanced customer queries. It not only retrieves suitable shoes but also explains its reasoning, making the interaction more interactive and customer-friendly.


## Conclusion: Day 1 - How AI Agents Improve Naive Chatbots by Asking Clarifying Questions

Today, we explored one of the most common limitations of naive Retrieval-Augmented Generation (RAG) systems: their inability to handle ambiguous queries effectively.  

Through our example of searching for shoes for a black-tie event, we demonstrated how:
1. Naive chatbots fall short by blindly vectorizing the entire user query and relying solely on embedding similarity.
2. AI agents enhance the experience by:
   - Interpreting the intent behind user queries
   - Asking clarifying questions to fill in missing details
   - Applying metadata filters to refine search results
   - Generating responses that are both accurate and customer-centric

By focusing on **contextual reasoning and dynamic interaction**, AI agents transform customer queries into meaningful, actionable results.

### What's Next?

Tomorrow, we'll continue our journey by tackling another challenge faced by naive chatbots.

---

### Ready to implement this in your own systems?  
AI agents are the future of e-commerce chatbots. If you're interested in applying these concepts to your business, feel free to reach out!
